# Pretrained DinoV2 model

In [1]:
%load_ext autoreload
%autoreload 2

Make directory to store the models

In [2]:
# ! mkdir -p /mnt/data/models
# %cd /mnt/data/models

/mnt/data/models


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


copy the file from GCS to `/mnt/data/models` directory

In [ ]:
# ! gsutil cp gs://dsgt-clef-plantclef-2024/data/models/PlantNet_PlantCLEF2024_pretrained_models_on_the_flora_of_south-western_europe.tar /mnt/data/models/

Unzip the `.tar` file

In [ ]:
# ! tar -xvf /mnt/data/models/PlantNet_PlantCLEF2024_pretrained_models_on_the_flora_of_south-western_europe.tar -C /mnt/data/models

use pretrained model for inference

In [ ]:
from argparse import ArgumentParser
import pandas as pd
from urllib.request import urlopen
from PIL import Image
import timm
import torch


def load_class_mapping(class_list_file):
    with open(class_list_file) as f:
        class_index_to_class_name = {i: line.strip() for i, line in enumerate(f)}
    return class_index_to_class_name


def load_species_mapping(species_map_file):
    df = pd.read_csv(species_map_file, sep=";", quoting=1, dtype={"species_id": str})
    df = df.set_index("species_id")
    return df["species"].to_dict()


def main(image_url, class_mapping, species_mapping, pretrained_path):
    cid_to_spid = load_class_mapping(class_mapping)
    spid_to_sp = load_species_mapping(species_mapping)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = timm.create_model(
        "vit_base_patch14_reg4_dinov2.lvd142m",
        pretrained=False,
        num_classes=len(cid_to_spid),
        checkpoint_path=pretrained_path,
    )
    model = model.to(device)
    model = model.eval()

    # get model specific transforms (normalization, resize)
    data_config = timm.data.resolve_model_data_config(model)
    transforms = timm.data.create_transform(**data_config, is_training=False)

    img = None
    if "https://" in image_url or "http://" in image_url:
        img = Image.open(urlopen(image_url))
    elif image_url != None:
        img = Image.open(image_url)

    if img != None:
        img = transforms(img).unsqueeze(0)
        img = img.to(device)
        output = model(img)  # unsqueeze single image into batch of 1
        top5_probabilities, top5_class_indices = torch.topk(
            output.softmax(dim=1) * 100, k=5
        )
        top5_probabilities = top5_probabilities.cpu().detach().numpy()
        top5_class_indices = top5_class_indices.cpu().detach().numpy()

        for proba, cid in zip(top5_probabilities[0], top5_class_indices[0]):
            species_id = cid_to_spid[cid]
            species = spid_to_sp[species_id]
            print(species_id, species, proba)

In [ ]:
# Define your paths and image URL directly
path = "/mnt/data/models/pretrained_models"
image_url = "https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/test/1361687/bd2d3830ac3270218ba82fd24e2290becd01317c.jpg"
class_mapping_file = f"{path}/class_mapping.txt"
species_mapping_file = f"{path}/species_id_to_name.txt"
model_path = "vit_base_patch14_reg4_dinov2_lvd142m_pc24_onlyclassifier_then_all"
pretrained_path = f"{path}/{model_path}/model_best.pth.tar"

main(image_url, class_mapping_file, species_mapping_file, pretrained_path)